In [1]:
import os 
import glob
import boto3
import sagemaker
from dotenv import load_dotenv
from time import strftime
from sagemaker import get_execution_role, image_uris
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput, Processor
load_dotenv()


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


True

In [2]:
import time
from sagemaker.workflow.parameters import (
    ParameterInteger,
    ParameterString,
    ParameterFloat
)

In [3]:
sess = sagemaker.Session()
bucket = 'sagemaker-practice-bucket-nuga'
role = os.environ['SM_ROLE']
region = sess.boto_session.region_name
sm = boto3.Session().client(service_name='sagemaker', region_name=region)

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


In [66]:
sess.upload_data(
    path='preprocess_script.py',
    bucket=bucket,
    key_prefix='fitness_project/code'
)

's3://sagemaker-practice-bucket-nuga/fitness_project/code/preprocess_script.py'

##### ENV PARAMETERS

In [10]:
!aws s3 ls 

2023-06-25 20:53:00 kinesisbucketnugatest
2023-10-24 18:08:11 sagemaker-eu-north-1-928389782162
2023-08-07 16:10:59 sagemaker-practice-bucket-nuga
2023-08-07 16:32:15 sagemaker-studio-v9s6knew9ua
2023-08-07 16:22:16 sagemaker-studio-xv6zxh7iw6m
2023-08-07 16:35:19 sagemaker-us-east-1-928389782162


In [39]:
container_base_path = "/opt/ml/processing"
s3_input_data_uri = f"s3://{bucket}/fitness_project/data/fitness_class_2212.csv"
data_output_path = f"s3://{bucket}/fitness_project/data"

In [40]:
code_uri = f"s3://{bucket}/fitness_project/code/preprocess_script.py"

In [41]:
# exp_name = ParameterString(
#     name='ExperimentName',
#     default_value=pipeline_name
# )

# input_data = ParameterString(
#     name='InputData',
#     default_value=s3_input_data_uri
# )

instance_type = ParameterString(
    name='InstanceType',
    default_value='ml.t3.medium'
)

instance_count = ParameterInteger(
    name='InstanceCount',
    default_value=1
)

In [67]:
processor = SKLearnProcessor(
    framework_version='0.23-1',
    role=role,
    instance_type='ml.t3.medium',
    instance_count=1,
    env={'AWS_DEFAULT_REGION':region}    
)

input_data = [
    ProcessingInput(
    input_name='input_data', 
    source=s3_input_data_uri,  
    destination=f'{container_base_path}/data'
),
    ProcessingInput(
    input_name='input_code', 
    source=code_uri,  
    destination=f'{container_base_path}/code'
)
]

output_data = [
    ProcessingOutput(
    source=f'{container_base_path}/train',  
    destination=f'{data_output_path}/train',  
    output_name='output_train_data' 
),
    ProcessingOutput(
    source=f'{container_base_path}/test',  
    destination=f'{data_output_path}/test',  
    output_name='output_test_data' 
)
]

INFO:botocore.credentials:Found credentials in environment variables.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


INFO:sagemaker.image_uris:Defaulting to only available Python version: py3


In [50]:
processor.run(
    job_name=f'test-processor{int(time.time())}',
    code='preprocess_script.py',
    inputs=input_data,
    outputs=output_data,
    arguments=['--sagemaker']
)

INFO:sagemaker:Creating processing-job with name test-processor1698243078


................................................................sagemaker



In [60]:
processor.jobs[-1].describe()['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']

's3://sagemaker-practice-bucket-nuga/fitness_project/data/train'

##### TRACKING 

In [4]:
import smexperiments
from smexperiments.trial import Trial
from smexperiments.tracker import Tracker
from smexperiments.experiment import Experiment
from smexperiments.trial_component import TrialComponent

In [42]:
experiment = Experiment.create(
    experiment_name=f'test-training-{int(time.time())}',
    description='Testing SKLearnProcessor',
    sagemaker_boto_client=sm
)

# trial = Trial.create(
#     trial_name=f'trial-{timestamp}',
#     experiment_name=pipeline_name,
#     sagemaker_boto_client=sm
# )

In [ ]:
experiment_config_process = {
    'ExperimentName': pipeline_name,
    'TrialName': trial.trial_name,
    'TrialComponentDisplayName': 'Process'
}

In [43]:
experiment.experiment_name

'test-training-1698283047'

In [69]:
for ratio in ['0.1', '0.2', '0.3']:
    
    trial = Trial.create(
    trial_name=f'trial-{int(time.time())}',
    experiment_name=experiment.experiment_name,
    sagemaker_boto_client=sm
)
    processor.run(
    job_name=f'test-processor{int(time.time())}',
    code='preprocess_script.py',
    inputs=input_data,
    outputs=output_data,
    arguments=[
        '--sagemaker',
        '--ratio',
        ratio
    ],
    experiment_config={
    'TrialName': trial.trial_name,
    'TrialComponentDisplayName': 'Process'
}
)
    


INFO:sagemaker:Creating processing-job with name test-processor1698245401


.........................................................................sagemaker



INFO:sagemaker:Creating processing-job with name test-processor1698246228


..................................................................
sagemaker


INFO:sagemaker:Creating processing-job with name test-processor1698246950


........................................................................
sagemaker


In [17]:
from sagemaker.analytics import ExperimentAnalytics

In [71]:
experiment_analytics = ExperimentAnalytics(
    experiment_name=experiment.experiment_name
)
experiment_analytics.dataframe()

INFO:botocore.credentials:Found credentials in environment variables.


sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


,TrialComponentName,DisplayName,SourceArn,AWS_DEFAULT_REGION,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,SageMaker.ImageUri - MediaType,SageMaker.ImageUri - Value,code - MediaType,...,input_code - MediaType,input_code - Value,input_data - MediaType,input_data - Value,output_test_data - MediaType,output_test_data - Value,output_train_data - MediaType,output_train_data - Value,Trials,Experiments
0,test-processor1698246950-aws-processing-job,Process,arn:aws:sagemaker:us-east-1:928389782162:proce...,us-east-1,1.0,ml.t3.medium,30.0,None,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,None,...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,[trial-1698246949],[test-processor-1698244803]
1,test-processor1698246228-aws-processing-job,Process,arn:aws:sagemaker:us-east-1:928389782162:proce...,us-east-1,1.0,ml.t3.medium,30.0,None,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,None,...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,[trial-1698246228],[test-processor-1698244803]
2,test-processor1698245401-aws-processing-job,Process,arn:aws:sagemaker:us-east-1:928389782162:proce...,us-east-1,1.0,ml.t3.medium,30.0,None,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,None,...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,[trial-1698245401],[test-processor-1698244803]


In [7]:
# trainpath=processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][0]['S3Output']['S3Uri']
trainpath='s3://sagemaker-practice-bucket-nuga/fitness_project/data/train'
trainpath

's3://sagemaker-practice-bucket-nuga/fitness_project/data/train'

In [8]:
# testpath=processor.latest_job.describe()['ProcessingOutputConfig']['Outputs'][1]['S3Output']['S3Uri']
testpath='s3://sagemaker-practice-bucket-nuga/fitness_project/data/test'
testpath

's3://sagemaker-practice-bucket-nuga/fitness_project/data/test'

##### TRAINING

In [9]:
from sagemaker.sklearn.estimator import SKLearn

In [47]:
sess.upload_data(
    path='training_script.py',
    bucket=bucket,
    key_prefix='fitness_project/code'
)

's3://sagemaker-practice-bucket-nuga/fitness_project/code/training_script.py'

In [153]:
training_dir = f"s3://{bucket}/fitness_project/code"

In [10]:
metric_definition = [
    {'Name': 'Accuracy', 'Regex': 'Accuracy: ([0-9\\.]+)'},
    {'Name': 'Precision', 'Regex': 'Precision: ([0-9\\.]+)'},
    {'Name': 'Recall', 'Regex': 'Recall: ([0-9\\.]+)'},
    {'Name': 'F1', 'Regex': 'F1: ([0-9\\.]+)'}
]

In [13]:
inputs = {
    "train": trainpath,
    "test": testpath
}

In [22]:
trial = Trial.create(
    trial_name=f'trial-{int(time.time())}',
    experiment_name=experiment.experiment_name,
    sagemaker_boto_client=sm
)

In [23]:
experiment.experiment_name

'test-training-1698275683'

In [24]:
from sagemaker.experiments import Run

In [48]:
with Run(experiment_name=experiment.experiment_name, run_name=f'check-run-{int(time.time())}') as run:
    
    estimator_parameters = {
    "entry_point": "training_script.py",
    # "source_dir": training_dir,
    "framework_version": "0.23-1",
    "py_version": "py3",
    "instance_type": "ml.m5.large",
    "instance_count": 1,
    # "hyperparameters": hyperparameters,
    "role": role,
    "base_job_name": "linearregression-model",
    "use_spot_instances": True,
    "max_wait": 7200,
    "max_run": 3600,
    "metric_definition": metric_definition
}

    estimator = SKLearn(**estimator_parameters) 

    estimator.fit(
        inputs,
        job_name=f'test-training{int(time.time())}',
    #     experiment_config={
    #     'TrialName': trial.trial_name,
    #     'TrialComponentDisplayName': 'Training'
    # },
    )

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml
Using provided s3_resource


INFO:sagemaker:Creating training-job with name: test-training1698284057


2023-10-26 01:34:21 Starting - Starting the training job...
2023-10-26 01:34:36 Starting - Preparing the instances for training......
2023-10-26 01:35:52 Downloading - Downloading input data...
2023-10-26 01:36:27 Training - Downloading the training image...
2023-10-26 01:36:58 Training - Training image download completed. Training in progress..2023-10-26 01:37:01,233 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2023-10-26 01:37:01,238 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-26 01:37:01,290 sagemaker_sklearn_container.training INFO     Invoking user training script.
2023-10-26 01:37:01,478 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-26 01:37:01,492 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-26 01:37:01,506 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2023-10-26 01:

ClientError: An error occurred (ValidationException) when calling the UpdateTrialComponent operation: 2 validation errors detected: Value 'Error for Training job test-training1698284057: Failed. Reason: AlgorithmError: framework error: 
Traceback (most recent call last):
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_containers/_trainer.py", line 84, in train
    entrypoint()
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 39, in main
    train(environment.Environment())
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_sklearn_container/training.py", line 35, in train
    runner_type=runner.ProcessRunnerType)
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/entry_point.py", line 100, in run
    wait, capture_error
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_training/process.py", line 291, in run
    cwd=environment.code_dir,
  File "/miniconda3/lib/python3.7/site-packages/sagemaker_traini

In [18]:
experiment_analytics = ExperimentAnalytics(
    experiment_name=experiment.experiment_name
)
experiment_analytics.dataframe()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/Labi/Library/Application Support/sagemaker/config.yaml


,TrialComponentName,DisplayName,SourceArn,SageMaker.ImageUri,SageMaker.InstanceCount,SageMaker.InstanceType,SageMaker.VolumeSizeInGB,sagemaker_container_log_level,sagemaker_job_name,sagemaker_program,...,test - MediaType,test - Value,train - MediaType,train - Value,SageMaker.DebugHookOutput - MediaType,SageMaker.DebugHookOutput - Value,SageMaker.ModelArtifact - MediaType,SageMaker.ModelArtifact - Value,Trials,Experiments
0,test-training1698275776-aws-training-job,Training,arn:aws:sagemaker:us-east-1:928389782162:train...,683313688378.dkr.ecr.us-east-1.amazonaws.com/s...,1.0,ml.m5.large,30.0,20.0,"""test-training1698275776""","""training_script.py""",...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-practice-bucket-nuga/fitness_pr...,None,s3://sagemaker-us-east-1-928389782162/,None,s3://sagemaker-us-east-1-928389782162/test-tra...,[trial-1698275771],[test-training-1698275683]


In [ ]:
!aws s3 cp --quite s3://{bucket}/fitness_project/data/train/

In [55]:
!aws s3 ls s3://{bucket}/fitness_project/data/train

                           PRE train/
